In [ ]:
import pandas as pd
import numpy as np

# **1) LOAD TITANIC DATASET**

In [ ]:
df= pd.read_csv('/content/titanic (1).csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df.describe(include='int')

,PassengerId,Survived,Pclass,SibSp,Parch
count,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,0.523008,0.381594
std,257.353842,0.486592,0.836071,1.102743,0.806057
min,1.000000,0.000000,1.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,0.000000,0.000000
75%,668.500000,1.000000,3.000000,1.000000,0.000000
max,891.000000,1.000000,3.000000,8.000000,6.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

# **2) SPLITING THE DATASET**

In [ ]:
X= df[['Pclass','SibSp','Parch', 'Fare']]
y=df['Survived']

SCALE DOWN THE FARE COLUMN

In [ ]:
X.head()

,Pclass,SibSp,Parch,Fare
0,3,1,0,7.2500
1,1,1,0,71.2833
2,3,0,0,7.9250
3,1,1,0,53.1000
4,3,0,0,8.0500


In [ ]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
X.loc[:,'Fare']=scaler.fit_transform(X[['Fare']])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X,y, test_size= 0.2, random_state=42)

# **3) SVM**

In [ ]:
from sklearn.svm import SVC

In [ ]:
clf= SVC(kernel='rbf')
clf.fit(X_train,y_train)
pred= clf.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(pred,y_test))

0.7206703910614525


# **4) 5 fold cross validation**

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
score= cross_val_score(clf,X_train,y_train,cv=5)
print(score)
print(np.average(score))

[0.69230769 0.69230769 0.66901408 0.72535211 0.64788732]
0.6853737811484291


# **5) RANDOM SEARCH AND HYPEROPT**

### **RANDOM SEARCH**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
grid= RandomizedSearchCV(clf,param_grid,cv=5, scoring="accuracy",verbose=1,n_jobs=-1, 
                             n_iter=50, random_state = 0)
grid.fit(X_train,y_train)
print(grid.best_params_)
print(grid.best_score_)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 25 is smaller than n_iter=50. Running 25 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'kernel': 'rbf', 'gamma': 1, 'C': 10}
0.7134640007879444


### **HYPER OPT**

In [ ]:
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials

In [ ]:
space = {
    "C": hp.choice("C", [0.1, 1, 10, 100, 1000]),
    "gamma": hp.choice("gamma", [1, 0.1, 0.01, 0.001, 0.0001])
}

In [ ]:
def hyperparameter_tuning(params):
    svcr = SVC(**params)
    acc = cross_val_score(svcr, X, y,scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [ ]:
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=50, 
    trials=trials
)

print("Best: {}".format(best))

100%|██████████| 50/50 [00:11<00:00,  4.54it/s, best loss: -0.7127863913125354]
Best: {'C': 4, 'gamma': 0}
